### ECCOv4r4 DAILY - select the pressure level to keep, or calculate the integral within 2 pressure levels

In [1]:
import numpy as np
import xarray as xr
import sys
import glob
import netCDF4 as nc
import os
import sys
## Import the ecco_v4_py library into Python
## =========================================
##    If ecco_v4_py is not installed in your local Python library, 
##    tell Python where to find it.  The example below adds
##    ecco_v4_py to the user's path if it is stored in the folder
##    ECCOv4-py under the user's home directory

from os.path import join,expanduser,exists,split
user_home_dir = expanduser('~')
sys.path.append(join(user_home_dir,'ECCOv4-py'))
import ecco_v4_py as ecco

# Suppress warning messages for a cleaner presentation
import warnings
warnings.filterwarnings('ignore')

In [2]:
# ECCOv4r4_DATA = xr.open_dataset('/Volumes/MyPassportForMac/MAC_15/NASA_project/2023/NEW_heatBudgetECCO_daily/data/outputs/nc_files_zlev_or_zint/ECCOv4r4_DATA_cut_zlev0_1992_2018.nc')

ECCOv4r4_DATA = xr.open_dataset('/Users/jacoposala/Desktop/CU/3.RESEARCH/NASA_project/heatBudgetECCO_jupyter/heat_nc_MONTHLY_files_for_donata/ECCOv4r4_DATA_1992_2017.nc')


### Inputs

In [3]:
# ECCO_dir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/NASA_project/NEW_heatBudgetECCO/data/'
ECCO_dir = '/Volumes/MyPassportForMac/MAC_15/NASA_project/2023/NEW_heatBudgetECCO_daily/data/outputs/'
save_dir = join(ECCO_dir)
integral_dir = '/Volumes/MyPassportForMac/MAC_15/NASA_project/2023/NEW_heatBudgetECCO_daily/data/outputs/'


In [9]:
year_start = 1992  # Define the start year
# year_end = 2018  # Define the end year (NOT included, will stop the year before this one)
year_end = 1994  # Define the end year (NOT included, will stop the year before this one)

In [10]:
# Choose between selection of a single level or integral
do_integral_tag = True
do_single_lev = True

### Select a single level and save a single file for every terms (for all the years)

In [12]:
if do_single_lev:
    z_flag = 'zlev'
    
    # Define the variables to be saved
    varnames = ['G_total_cut', 'G_advection_cut', 'G_diffusion_cut', 'G_forcing_cut',
                'adv_hConv_cut', 'adv_vConv_cut', 'dif_hConv_cut', 'dif_vConv_cut',
                'G_advection_conv_zon_cut', 'G_advection_conv_mer_cut',
                'G_diffusion_conv_zon_cut', 'G_diffusion_conv_mer_cut', 
                'DATA_cut']

    # need to do G_total for k5 and k9
    
    # Choose the vertical levels of interest
    z_levels = [0]#, 5, 9]  # 0, 5, 9, or any other desired levels
    
    # Loop through the vertical levels
    for z_lev in z_levels:
        print('Depth level: ', z_lev)
        # Loop through the variables
        for varname in varnames:
            print('Starting: ', varname)
            final_dataset = None  # Initialize final dataset for each variable

            # Loop through the years from 2004 to 2018
            for year in range(year_start, year_end):
                print(year)
                # Define the folder name for the current year
                folder_year = str(year)

                # Create the directory path for loading
                load_path = os.path.join(integral_dir, 'single_heat_terms', folder_year + '/')
                
                # Define the file name for loading
                load_file_name = f'ECCOv4r4_{varname}_{year}.nc'

                # Load the NetCDF file from the specified directory
                load_file_path = os.path.join(load_path, load_file_name)
                dataset = xr.open_dataset(load_file_path).sel(k=z_lev)
                
                # Check if the variable name contains "_conv" (files up to 2003 included do...)
                if any(var in list(dataset.variables.keys()) for var in ['G_advection_conv_cut', 'G_diffusion_conv_cut']):   
                    # If so, remove the _conv from the name
                    dataset = dataset.rename_vars({f'{varname[:-4]}_conv_cut': varname})
                
                # Ensure coordinates match the final dataset
                if final_dataset is None:
                    final_dataset = dataset
                else:
                    # Concatenate along the 'time' dimension
                    final_dataset = xr.concat([final_dataset, dataset], dim='time')

            # Save the final dataset for the current variable
            final_dataset.to_netcdf(save_dir + '/nc_files_zlev_or_zint/' + f'ECCOv4r4_{varname}_{z_flag}{z_lev}_{year_start}_{year_end}.nc', format='NETCDF4')


Depth level:  0
Starting:  G_total_cut
1992
1993
Starting:  G_advection_cut
1992
1993
Starting:  G_diffusion_cut
1992
1993
Starting:  G_forcing_cut
1992
1993
Starting:  adv_hConv_cut
1992
1993
Starting:  adv_vConv_cut
1992
1993
Starting:  dif_hConv_cut
1992
1993
Starting:  dif_vConv_cut
1992
1993
Starting:  G_advection_conv_zon_cut
1992
1993
Starting:  G_advection_conv_mer_cut
1992
1993
Starting:  G_diffusion_conv_zon_cut
1992
1993
Starting:  G_diffusion_conv_mer_cut
1992
1993
Starting:  DATA_cut
1992
1993


### Compute the integral for X levels and save a single file for every terms (for all the years)

In [ ]:
# The deepest ocean bottom is set to 6145m below the surface, with the vertical grid spacing increasing 
# from 10m near the surface to 457m near the ocean bottom.

In [13]:
if do_integral_tag:
    z_flag = 'zint'
    Z_depth = xr.open_dataset('/Users/jacoposala/Desktop/CU/3.RESEARCH/NASA_project/NEW_heatBudgetECCO/\
data/outputs/single_heat_terms/ECCOv4r4_Z_depth_1993_2017.nc').Z_depth

    expected_order = ('time', 'tile', 'k', 'j', 'i')

    varnames = ['G_total_cut', 'G_advection_cut', 'G_diffusion_cut', 'G_forcing_cut',
                'adv_hConv_cut', 'adv_vConv_cut', 'dif_hConv_cut', 'dif_vConv_cut',
                'G_advection_conv_zon_cut', 'G_advection_conv_mer_cut',
                'G_diffusion_conv_zon_cut', 'G_diffusion_conv_mer_cut', 
                'DATA_cut']

  
    for varname in varnames:
        print('Processing ', varname)
        print('Year being processed:')
        final_dataset = None  # Initialize final dataset for each variable

        for year in range(year_start, year_end):
            print(year)
            folder_year = str(year)
            load_path = os.path.join(integral_dir, 'single_heat_terms', folder_year)
            load_file_name = f'ECCOv4r4_{varname}_{year}.nc'
            load_file_path = os.path.join(load_path, load_file_name)
            dataset = xr.open_dataset(load_file_path)
            
            # Check if the variable name contains "_conv" (files up to 2003 included do...)
            if any(var in list(dataset.variables.keys()) for var in ['G_advection_conv_cut', 'G_diffusion_conv_cut']):   
                # If so, remove the _conv from the name
                dataset = dataset.rename_vars({f'{varname[:-4]}_conv_cut': varname})
                    
            # Fix the order of coordinates for G_forcing_cut, adv_vConv_cut, and dif_vConv_cut
            if varname in ['G_forcing_cut', 'adv_vConv_cut', 'dif_vConv_cut']:
                dataset = dataset.transpose('time', 'k', 'tile', 'j', 'i')
            
            if varname == 'DATA_cut':
                # Conversion to Kelvin for DATA_cut
                variable_cut = dataset[varname] + 273.15
            else:
                variable_cut = dataset[varname]
                        
            depth_index_top_integral = 0  # corresponds to 5m
            depth_index_bottom_integral = 5  # corresponds to 55m

            thickness = np.abs(np.diff(Z_depth))

            selected_variable_cut = variable_cut[:, depth_index_top_integral:depth_index_bottom_integral, :, :, :]
            selected_thickness = thickness[depth_index_top_integral:depth_index_bottom_integral - 1]
            reshaped_thickness_diff = np.reshape(selected_thickness, (1, -1, 1, 1, 1))

            variable_integral = (
                (selected_variable_cut[:, :-1, :, :, :].values + selected_variable_cut[:, 1:, :, :, :].values)
                * reshaped_thickness_diff
            ).sum(axis=1) / 2

            # Save the integral to a new NetCDF file
            save_path = os.path.join(save_dir + '/nc_files_zlev_or_zint/')
            save_file_name = f'integral_{varname}_{year}.nc'
            save_file_path = os.path.join(save_path, save_file_name)
            
            # Modify variable name and add _ohc_k0k5
            varname_modified = varname.replace('_cut', '') + f'_ohc_k{depth_index_top_integral}_k{depth_index_bottom_integral}'
        
            integral_dataset = xr.Dataset(
                {
                    varname_modified: (["time", "tile", "j", "i"], variable_integral)
                },
                coords={
                    "time": variable_cut.time.values,
                    "tile": np.arange(13),  # Adjust the range according to the number of tiles
                    "j": np.arange(90),  # Adjust the range according to the latitude dimensions
                    "i": np.arange(90),  # Adjust the range according to the longitude dimensions
                }
            )
            # Save files for single years
            #integral_dataset.to_netcdf(save_file_path)

            if final_dataset is None:
                final_dataset = integral_dataset
            else:
                final_dataset = xr.concat([final_dataset, integral_dataset], dim='time')

        # At the end of all years, save the concatenated file, too
        if final_dataset is not None:
            # Save the final dataset to a single NetCDF file
            final_save_path = os.path.join(save_path, f'ECCOv4r4_{varname_modified}_{year_start}_{year_end}.nc')
            
            final_dataset.to_netcdf(final_save_path)


Processing  G_total_cut
Year being processed:
1992
1993
Processing  G_advection_cut
Year being processed:
1992
1993
Processing  G_diffusion_cut
Year being processed:
1992
1993
Processing  G_forcing_cut
Year being processed:
1992
1993
Processing  adv_hConv_cut
Year being processed:
1992
1993
Processing  adv_vConv_cut
Year being processed:
1992
1993
Processing  dif_hConv_cut
Year being processed:
1992
1993
Processing  dif_vConv_cut
Year being processed:
1992
1993
Processing  G_advection_conv_zon_cut
Year being processed:
1992
1993
Processing  G_advection_conv_mer_cut
Year being processed:
1992
1993
Processing  G_diffusion_conv_zon_cut
Year being processed:
1992
1993
Processing  G_diffusion_conv_mer_cut
Year being processed:
1992
1993
Processing  DATA_cut
Year being processed:
1992
1993


In [ ]:
ciao

In [ ]:
# forcing = xr.open_dataset('/Volumes/MyPassportForMac/MAC_15/NASA_project/2023/NEW_heatBudgetECCO_daily/data/outputs/single_heat_terms/2006/ECCOv4r4_G_forcing_cut_2006.nc')
# forcing

In [ ]:
# data = xr.open_dataset('/Volumes/MyPassportForMac/MAC_15/NASA_project/2023/NEW_heatBudgetECCO_daily/data/outputs/single_heat_terms/2004/ECCOv4r4_DATA_cut_2004.nc')
# data


In [ ]:
test = xr.open_dataset('/Users/jacoposala/Downloads/ECCOv4r4_G_diffusion_cut_zlev0_1992_2018.nc')
test


In [ ]:
# if do_single_lev:
#     z_flag = 'zlev'
#     # Define the variables to be saved
#     varnames = ['G_total_cut', 'G_advection_cut', 'G_diffusion_cut', 'G_forcing_cut',
#                 'adv_hConv_cut', 'adv_vConv_cut', 'dif_hConv_cut', 'dif_vConv_cut',
#                 'G_advection_conv_zon_cut', 'G_advection_conv_mer_cut',
#                 'G_diffusion_conv_zon_cut', 'G_diffusion_conv_mer_cut', 
#                 'DATA_cut']
    
#     # Choose the vertical level of interest
#     z_lev = 9 # 0,5,9

#     # Loop through the variables
#     for varname in varnames:
#         print('Starting: ', varname)
#         final_dataset = None  # Initialize final dataset for each variable

#         # Loop through the years from 2004 to 2018
#         for year in range(year_start, year_end):
#             # Define the folder name for the current year
#             folder_year = str(year)

#             # Create the directory path for loading
#             load_path = os.path.join(integral_dir, 'single_heat_terms', folder_year)

#             # Define the file name for loading
#             load_file_name = f'ECCOv4r4_{varname}_{year}.nc'

#             # Load the NetCDF file from the specified directory
#             load_file_path = os.path.join(load_path, load_file_name)
#             dataset = xr.open_dataset(load_file_path).sel(k=z_lev)
            
            
#             # Ensure coordinates match the final dataset
#             if final_dataset is None:
#                 final_dataset = dataset
#             else:
#                 # Concatenate along the 'time' dimension
#                 final_dataset = xr.concat([final_dataset, dataset], dim='time')

#         # Save the final dataset for the current variable
#         final_dataset.to_netcdf(save_dir + '/nc_files_zlev_or_zint/' + f'ECCOv4r4_{varname}_{z_flag}{z_lev}_{year_start}_{year_end}.nc', format='NETCDF4')
